# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Read CSV
cities_df=pd.read_csv("output_data.csv")
cities_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,halvad,23.0167,71.1833,92.52,21,0,8.75,IN,1619366012
1,1,basco,20.4487,121.9702,75.94,75,100,17.05,PH,1619366013
2,2,kapaa,22.0752,-159.3190,73.40,78,75,13.80,US,1619366013
3,3,kununurra,-15.7667,128.7333,60.80,88,70,1.14,AU,1619366013
4,4,geraldton,-28.7667,114.6000,62.60,82,0,3.44,AU,1619366014


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)

In [13]:
# Configure heat map
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 7)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
vacation_df=cities_df.loc[(cities_df["Max Temp"]<85)&(cities_df["Max Temp"]>=75)].dropna()
vacation_df

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,basco,20.4487,121.9702,75.94,75,100,17.05,PH,1619366013
7,7,hithadhoo,-0.6000,73.0833,83.41,66,47,4.52,MV,1619366015
8,8,vallenar,-28.5708,-70.7581,75.27,36,0,4.09,CL,1619366015
9,9,rikitea,-23.1203,-134.9692,77.32,75,80,7.85,PF,1619365928
10,10,iquitos,-3.7481,-73.2472,82.40,78,75,6.91,PE,1619365973
...,...,...,...,...,...,...,...,...,...,...
508,508,santa rosalia,17.4860,120.4348,75.00,74,0,4.29,PH,1619366064
509,509,ambon,-3.6954,128.1814,81.00,88,92,8.63,ID,1619366160
512,512,praia,14.9215,-23.5087,78.80,50,40,21.85,CV,1619366146
515,515,aden,12.7794,45.0367,82.76,82,76,16.62,YE,1619366161


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df=vacation_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"]=""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,basco,PH,20.4487,121.9702,
7,hithadhoo,MV,-0.6000,73.0833,
8,vallenar,CL,-28.5708,-70.7581,
9,rikitea,PF,-23.1203,-134.9692,
10,iquitos,PE,-3.7481,-73.2472,
...,...,...,...,...,...
508,santa rosalia,PH,17.4860,120.4348,
509,ambon,ID,-3.6954,128.1814,
512,praia,CV,14.9215,-23.5087,
515,aden,YE,12.7794,45.0367,


In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():


    Latitude=row['Latitude']
    Longitude=row['Longitude']

    # add keyword to params dict
    params['location'] = f'{Latitude},{Longitude}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 1
------------
Retrieving Results for Index 7
------------
Retrieving Results for Index 8
------------
Retrieving Results for Index 9
------------
Retrieving Results for Index 10
------------
Retrieving Results for Index 16
------------
Retrieving Results for Index 22
------------
Retrieving Results for Index 27
------------
Retrieving Results for Index 28
------------
Retrieving Results for Index 33
Missing field/result... skipping.
------------
Retrieving Results for Index 37
------------
Retrieving Results for Index 39
------------
Retrieving Results for Index 49
------------
Retrieving Results for Index 52
------------
Retrieving Results for Index 54
------------
Retrieving Results for Index 56
------------
Retrieving Results for Index 64
------------
Retrieving Results for Index 78
------------
Retrieving Results for Index 80
------------
Retrieving Results for Index 84
------------
Retrieving Results for Index 85
------------
Retrieving Results for In

In [22]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,basco,PH,20.4487,121.9702,Batanes Seaside Lodge Annex
7,hithadhoo,MV,-0.6000,73.0833,Sahibaan
8,vallenar,CL,-28.5708,-70.7581,Humacao Bed & Breakfast
9,rikitea,PF,-23.1203,-134.9692,Pension Maro'i
10,iquitos,PE,-3.7481,-73.2472,Marfil Del Amazonas MDA
...,...,...,...,...,...
508,santa rosalia,PH,17.4860,120.4348,The Mansion
509,ambon,ID,-3.6954,128.1814,Airy Lapangan Merdeka Ambon Wem Reawaru 7B
512,praia,CV,14.9215,-23.5087,Hotel Santa Maria
515,aden,YE,12.7794,45.0367,محلات فدعق التجارية للاقمشة والجلابيات


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [27]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))